In [1]:
from fingerprint import *

In [2]:
# set path to a file or directory to add it to the database

path = './test_songs_2/BTS/'
add_song(path)

skipping 01 Converse High.mp3
skipping 08 I Need U (Remix).mp3
skipping 06 Love Is Not Over (Full Length Edition).mp3
skipping 06 Run.mp3
skipping 05 House Of Cards (Full Length Edition).mp3
skipping 11 Butterfly (Alternative Mix).mp3
skipping 05 Butterfly (Prologue Mix).mp3
skipping 10 Run (Alternative Mix).mp3
skipping 03 Hold Me Tight.mp3
skipping 02 Moving On.mp3
skipping 02 I Need U.mp3
skipping 01 Intro _ The Most Beautiful Moment in Life.mp3
skipping 07 I Need U (Urban Mix).mp3
skipping 09 Dope.mp3
skipping 08 Silver Spoon.mp3
skipping 04 Autumn Leaves.mp3
skipping 11 Save Me.mp3
skipping 09 Run (Ballad Mix).mp3
skipping 07 Ma City.mp3
skipping cover.jpg
skipping 04 Butterfly.mp3
skipping 10 Fire.mp3
skipping 12 Epilogue_ Young Forever.mp3
skipping 03 Whalien 52.mp3
skipping 02 DNA.m4a
skipping 06 Billboard Music Awards Speech [Skit].m4a
skipping 07 MIC Drop.m4a
skipping 08 Go Go.m4a
skipping 03 Best Of Me.m4a
skipping 11 Sea.m4a
skipping 10 Hesitation and Fear[Skit].m4a
skippin